# Chapter 04: Use of OntoGPT

Ray LeClair \<2024-07-08 Mon\>

## Objectives

Since the NCBI Cell pipeline calls for the use of a Large Language
Model (LLM) to process publications associated with CELLxGENE
datasets, the objective of this document is to:

-   Investigate the use of OntoGPT for publication processing

OntoGP is a Python package for extracting structured information from
text using LLMs, instruction prompts, and ontology-based grounding.

See: [OntoGPT](https://monarch-initiative.github.io/ontogpt/)

### Background

The primary extraction method currently implemented in OntoGPT is
SPIRES: Structured Prompt Interrogation and Recursive Extraction of
Semantics, which:

-   Implements a Zero-Shot Learning (ZSL) approach to extracting nested
    semantic structures from text

-   Uses OpenAI GPT models through their API

This demonstration use the `pubmed-annotate` function to retrieve full
text of a publicaiton given a PubMed identifier (PMID), then performs
extraction on the text with SPIRES using the "Cell Type" template.

The following sections describe various development environments

See:

-   [SPIRES: Structured Prompt Interrogation and Recursive Extraction of Semantics](https://academic.oup.com/bioinformatics/article/40/3/btae104/7612230)
-   [pubmed-annotate](https://monarch-initiative.github.io/ontogpt/functions/#pubmed-annotate)
-   [Cell Type](https://monarch-initiative.github.io/ontogpt/cell_type/)
-   [springbok-nlm-kn/README.md](https://github.com/ralatsdc/springbok-nlm-kn/blob/main/README.md)

### Jupyter Notebook development environment

Launch Jupyter Notebook from a terminal in which `.zshenv` has been
sourced, and the virtual environment has been activated.

### Emacs Org Mode development environment

Launch Emacs from a terminal in which `.zshenv` has been sourced, then
evaluate this code block to activate the virtual environment:

``` commonlisp
(pyvenv-activate "../../.venv")
```

## Investigate the use of OntoGPT for publication processing

To begin, we import modules, and assign module scope variables:

In [1]:
import os
import subprocess
from traceback import print_exc

DATA_DIR = "../data"

ONTOGPT_DIR = f"{DATA_DIR}/ontogpt"


Then recall that in [Chapter-02-E-Utilities.org](Chapter-02-E-Utilities.org) we saw how to get the
PMID for a title:

In [2]:
from CELLxGENE import get_metadata_and_datasets, get_title
from E_Utilities import get_pmid_for_title
try:
    datasets, counts, var, obs = get_metadata_and_datasets()
    citation = datasets["citation"].iloc[4]
    title = get_title(citation)
    pmid = get_pmid_for_title(title)
except Exception:
    print_exc()
print(f"PMID: {pmid} found for title: {title}")


Reading datasets parquet
Reading summary cell counts parquet
Reading gene metadata parquet
Reading cell metadata parquet
Getting title for citation URL: https://doi.org/10.1038/s41467-022-29450-x
Trying requests
Found title: 'Single-cell Atlas of common variable immunodeficiency shows germinal center-associated epigenetic dysregulation in B-cell responses' for citation URL: https://doi.org/10.1038/s41467-022-29450-x
Getting PMID for title: 'Single-cell Atlas of common variable immunodeficiency shows germinal center-associated epigenetic dysregulation in B-cell responses'
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
{'db': 'pubmed', 'term': 'Single-cell Atlas of common variable immunodeficiency shows germinal center-associated epigenetic dysregulation in B-cell responses', 'field': 'title', 'retmode': 'json', 'email': 'raymond.leclair@gmail.com', 'api_key': '10a233bf53a1db01e78bc08e59520ace5e09'}
Found PMID: 35365635 for title: 'Single-cell Atlas of common variable immunod

Since the `pubmed-annotate` function of OntoGPT is run on the command
line, we use Python's `subprocess` module, caching results in a file
to prevent duplicate processing:

In [3]:
def run_ontogpt_pubmed_annotate(pmid):
    """Run the OntoGPT pubmed-annotate function for the specified PMID
    associated with a dataset.

    Parameters
    ----------
    pmid : str
       The PubMed identifier found

    Returns
    -------
    None
    """
    # Run OntoGPT pubmed-annotate function, if needed
    if pmid is None:
        return
    output_filename = f"{pmid}.out"
    output_filepath = f"{ONTOGPT_DIR}/{output_filename}"
    if not os.path.exists(output_filepath):
        print(f"Running ontogpt pubmed-annotate for PMID: {pmid}")
        subprocess.run(
            [
                "ontogpt",
                "pubmed-annotate",
                "--template",
                "cell_type",
                pmid,
                "--limit",
                "1",
                "--output",
                output_filepath,
            ],
        )
        print(f"Completed ontogpt pubmed-annotate for PMID: {pmid}")

    else:
        print(f"Ontogpt pubmed-annotate output for PMID: {pmid} exists")


Now call the function with the PMID obtained earler:

In [4]:
try:
    run_ontogpt_pubmed_annotate(pmid)
except Exception:
    print_exc()


Ontogpt pubmed-annotate output for PMID: 35365635 exists


Next, in Chapter 05 we'll use the results produced by NS-Forest to
populate an ArangoDB database graph.

See:

-   [Chapter-05-ArangoDB.ipynb](Chapter-05-ArangoDB.ipynb)
-   [Chapter-03-NS-Forest.ipynb](Chapter-03-NS-Forest.ipynb)